In [52]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
import os

In [53]:
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
os.environ["OPENAI_API_KEY"] = "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"

model = ChatOpenAI(model="deepseek/deepseek-r1:free")

In [54]:
# create state
class promt_chaining_llm(TypedDict):

    tittle : str
    outline : str
    content : str

In [55]:
# create graph
graph = StateGraph(promt_chaining_llm)

In [56]:
def create_outline(state : promt_chaining_llm) -> promt_chaining_llm:

    # fetch title
    title = state['tittle']

    # create the task prompt
    prompt = f'generate a small outline for the title for blog. {title}'

    # call the llm generative model
    created_outline = model.invoke(prompt).content

    # update the outline state
    state['outline'] = created_outline

    return state

In [57]:
def create_blog(state : promt_chaining_llm) -> promt_chaining_llm:

    # fetch updated title and outline
    tittle = state['tittle']
    updated_outline = state['outline'] 

    # create the task prompt
    prompt = f'generate a small blog of 1000 words for this title. {updated_outline} using the following outline. \n {updated_outline}'

    # call the llm generative model
    updated_content = model.invoke(prompt).content

    # update the content state
    state['content'] = updated_content

    return state


In [58]:
# add node
graph.add_node('create outline', create_outline)
graph.add_node('create blog', create_blog)

# add edges
graph.add_edge(START, 'create outline')
graph.add_edge('create outline' , 'create blog')
graph.add_edge('create blog', END)

# compile graph
prompt_chaining_workflow = graph.compile()

In [59]:
# execute the workflow
initial_state = {'tittle': "history of India"}
final_state = prompt_chaining_workflow.invoke(initial_state)
print(final_state)


KeyboardInterrupt: 